# Panorama Downloader for Kaggle

This notebook downloads panorama images from Google Street View based on a `panoids.json` file.

## 1. Setup

First, let's install the necessary libraries.

In [ ]:
!pip install aiohttp==3.6.3 async-timeout==3.0.1 attrs==20.2.0 branca==0.4.1 certifi==2020.6.20 chardet==3.0.4 folium==0.11.0 idna==2.10 Jinja2==2.11.2 MarkupSafe==1.1.1 multidict==4.7.6 numpy==1.19.2 Pillow==8.0.0 requests==2.24.0 urllib3==1.25.11 yarl==1.5.1 PyYAML==5.3.1

Now, let's import the libraries we'll need.

In [1]:
import asyncio\n
import json\n
import os\n
import traceback\n
import glob\n
import aiohttp\n
import folium\n
from IPython.display import display, HTML\n
from PIL import Image\n
import itertools

SyntaxError: unexpected character after line continuation character (1700840424.py, line 1)

## 2. Configuration

Please upload your `panoids.json` file to the root directory of the Kaggle environment.

In [ ]:
PANOIDS_JSON_PATH = 'panoids.json'\n
OUTPUT_DIR = '/kaggle/working/'\n
PATH_TILES = os.path.join(OUTPUT_DIR, 'tiles')\n
PATH_PANORAMAS = os.path.join(OUTPUT_DIR, 'panoramas')\n
os.makedirs(PATH_TILES, exist_ok=True)\n
os.makedirs(PATH_PANORAMAS, exist_ok=True)

## 3. Visualize Panorama Locations

In [ ]:
if not os.path.exists(PANOIDS_JSON_PATH):\n
    print('panoids.json not found. Please upload the file.')\n
else:\n
    with open(PANOIDS_JSON_PATH, 'r') as f:\n
        panoids = json.load(f)\n
    print(f'Loaded {len(panoids)} panoids.')\n
\n
    if panoids:\n
        # Calculate the center of the map\n
        avg_lat = sum(p['lat'] for p in panoids) / len(panoids)\n
        avg_lon = sum(p['lon'] for p in panoids) / len(panoids)\n
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)\n
\n
        # Add a marker for each panorama\n
        for pan in panoids:\n
            folium.CircleMarker([pan['lat'], pan['lon']], popup=pan['panoid'], radius=1, color='blue', fill=True).add_to(m)\n
\n
        # Save the map to an HTML file\n
        map_path = os.path.join(OUTPUT_DIR, 'panorama_locations.html')\n
        m.save(map_path)\n
\n
        # Display the map in the notebook\n
        display(m)\n
    else:\n
        print('No panoids to display.')

## 4. Panorama Download Functions

In [ ]:
imgx = 26\n
\n
def tiles_info(panoid):\n
    image_url = 'http://cbk0.google.com/cbk?output=tile&panoid={0:}&zoom=5&x={1:}&y={2:}'\n
    coord = list(itertools.product(range(imgx), range(13)))\n
    tiles = [(x, y, '%s_%dx%d.jpg' % (panoid, x, y), image_url.format(panoid, x, y)) for x, y in coord]\n
    return tiles\n
\n
def stich_tiles(panoid, tiles, directory, final_directory, point=None):\n
    tile_width = 512\n
    tile_height = 512\n
    panorama = Image.new('RGB', (imgx*tile_width, 13*tile_height))\n
    for x, y, fname, url in tiles:\n
        fname = directory + '/' + fname\n
        tile = Image.open(fname)\n
        panorama.paste(im=tile, box=(x*tile_width, y*tile_height))\n
        del tile\n
    if point == None:\n
        name = f'{panoid}.jpg'\n
    else:\n
         name = f'{point[0]}_{point[1]}_{panoid}.jpg'\n
    panorama.save(os.path.join(final_directory, name), quality=30)\n
    del panorama\n
\n
def delete_tiles(tiles, directory):\n
    for x, y, fname, url in tiles:\n
        os.remove(directory + '/' + fname)

## 5. Download Panoramas

In [ ]:
async def download_tiles_async(tiles, directory, session):\n
    total_tiles = len(tiles)\n
    for i, (x, y, fname, url) in enumerate(tiles):\n
        url = url.replace('http://', 'https://')\n
        max_retries = 3\n
        retry_count = 0\n
        while retry_count < max_retries:\n
            try:\n
                async with session.get(url) as response:\n
                    if response.status == 200:\n
                        content = await response.read()\n
                        with open(directory + '/' + fname, 'wb') as out_file:\n
                            out_file.write(content)\n
                        print(f'\rDownloading tile {i+1}/{total_tiles}', end='', flush=True)\n
                        break\n
                    else:\n
                        print(f'\nFailed to download tile {fname}: HTTP {response.status}')\n
                        retry_count += 1\n
            except Exception as e:\n
                print(f'\nError downloading tile {fname}: {str(e)}')\n
                retry_count += 1\n
                if retry_count < max_retries:\n
                    print(f'Retrying... ({retry_count}/{max_retries})')\n
                    await asyncio.sleep(1)\n
        if retry_count >= max_retries:\n
            print(f'\nFailed to download tile {fname} after {max_retries} attempts')\n
\n
async def download_panorama(panoid, session):\n
    try:\n
        x = tiles_info(panoid['panoid'])\n
        await download_tiles_async(x, PATH_TILES, session)\n
        all_tiles_exist = all(os.path.exists(os.path.join(PATH_TILES, fname)) for _, _, fname, _ in x)\n
        if all_tiles_exist:\n
            try:\n
                stich_tiles(panoid['panoid'], x, PATH_TILES, PATH_PANORAMAS, point=(panoid['lat'], panoid['lon']))\n
            except Exception as e:\n
                print(f'Error during image stitching: {str(e)}')\n
                raise\n
        else:\n
            print(f'Some tiles are missing for panorama {panoid['panoid']}')\n
            raise Exception('Incomplete tile download')\n
        delete_tiles(x, PATH_TILES)\n
    except Exception as e:\n
        print(f'Failed to create panorama: {str(e)}\n{traceback.format_exc()}')\n
\n
def panoid_created(panoid):\n
    file = f'{panoid['lat']}_{panoid['lon']}_{panoid['panoid']}.jpg'\n
    return os.path.isfile(os.path.join(PATH_PANORAMAS, file))\n
\n
async def download_loop(panoids, pmax):\n
    conn = aiohttp.TCPConnector(limit=10)\n
    async with aiohttp.ClientSession(connector=conn, auto_decompress=False) as session:\n
        try:\n
            pending_panoids = [p for p in panoids[:pmax] if not panoid_created(p)]\n
            total = len(pending_panoids)\n
            if total == 0:\n
                print('No new panoramas to download in this batch')\n
                return\n
            print(f'\nProcessing {total} panoramas in this batch')\n
            for i, panoid in enumerate(pending_panoids, 1):\n
                try:\n
                    print(f'\nDownloading panorama {i}/{total} (ID: {panoid['panoid']})')\n
                    await download_panorama(panoid, session=session)\n
                except Exception as e:\n
                    print(f'Error processing panorama {panoid['panoid']}: {str(e)}')\n
                    continue\n
        except Exception as e:\n
            print(f'Error in download loop: {str(e)}')\n
            print(traceback.format_exc())\n

In [ ]:
async def main():\n
    if not os.path.exists(PANOIDS_JSON_PATH):\n
        print('panoids.json not found. Please upload the file.')\n
        return\n
    with open(PANOIDS_JSON_PATH, 'r') as f:\n
        panoids = json.load(f)\n
    print(f'Loaded {len(panoids)} panoids')\n
    i = 0\n
    try:\n
        while True:\n
            i += 1\n
            print(f'Running the next batch: {(i-1)*100+1} → {i*100}')\n
            await download_loop(panoids, 100 * i)\n
            if 100 * i > len(panoids):\n
                break\n
    except Exception as e:\n
        print(f'An error occurred: {e}')\n
\n
await main()